In [1]:
import sys
sys.path.append('../../robotics-course/build')
sys.path.append('../../robotics-course/scenarios/build')
import cv2 as cv
import numpy as np
import libry as ry
import time
# from perception import find_ball
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../robotics-course/scenarios/challenge.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
# you can also change the shape & size
targetObj=RealWorld.getFrame("obj0")
targetObj.setColor([1.,0,0])
# targetObj.setShape(ry.ST.sphere, [.03])
RealWorld.getFrame("obj0").setShape(ry.ST.ssBox, [.05, .05, .05, .01])
targetObj.setPosition([-0.1, .1, 0.7])
targetObj.setContact(1)

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
    print("deleting", name)
    RealWorld.delFrame(name)

V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

deleting obj1
deleting obj2
deleting obj3
deleting obj4
deleting obj5
deleting obj6
deleting obj7
deleting obj8
deleting obj9
deleting obj10
deleting obj11
deleting obj12
deleting obj13
deleting obj14
deleting obj15
deleting obj16
deleting obj17
deleting obj18
deleting obj19
deleting obj20
deleting obj21
deleting obj22
deleting obj23
deleting obj24
deleting obj25
deleting obj26
deleting obj27
deleting obj28
deleting obj29


In [4]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [5]:
# create your model world
C = ry.Config()
C.addFile('../../robotics-course/scenarios/pandasTable.g')
#V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

# save initial background
[rgb0, depth0] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
points0 = S.depthData2pointCloud(depth0, fxfypxpy)

In [6]:
obj = C.addFrame("object")
# obj.setShape(ry.ST.sphere, [.03])
obj.setShape(ry.ST.ssBox, [.05, .05, .05, .01])
obj.setColor([0,1,0])
obj.setContact(1)

V.setConfiguration(C)

In [ ]:
points = []
tau = .01


APPORACHING_OBJECT = 0
CLOSING_GRIPPER = 1
GOING_BACK = 2
RELEASING_OBJECT = 3

RSTATE = APPORACHING_OBJECT

def update(komo):
    komo.optimize(True)
    end_config = komo.getConfiguration(0)
    C.setFrameState(end_config)
    #RealWorld.setFrameState(end_config)
    V.setConfiguration(C)
    #return C.evalFeature(ry.FS.qItself, [])
    return C.getJointState()

def go_to_object(gripper, obj):
    komo = C.komo_path(1.,1,tau,True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper,obj], ry.OT.sos, [1e3]);
    #komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "percept"], ry.OT.sos, [1e2]);
    komo.addObjective([1.], ry.FS.vectorZ, [gripper], ry.OT.sos, [1e2], target=[0,0,1]);
    komo.addObjective([1.], ry.FS.scalarProductXZ, [obj,gripper], ry.OT.sos, [1e2]);
    komo.addObjective([1.], ry.FS.scalarProductXY, [obj,gripper], ry.OT.sos, [1e2]);
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
    #komo.addObjective([1.], ry.FS.qItself, [], ry.OT.sos, [1e3], order=1);
    komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
    return komo

def move_to_position(pos):
    komo = C.komo_path(1.,1,tau,True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
    komo.addObjective([1.], ry.FS.position, ["R_gripperCenter"], ry.OT.sos, [1e3], target=pos);
    #komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
    return komo
    
def start_grip_object(gripper):
    if not S.getGripperIsGrasping(gripper):
        S.closeGripper(gripper)

def grip_is_finished(gripper):
    return S.getGripperIsGrasping(gripper)

is_gripping = False
fall_delay = 0

for t in range(10000):
    
    print("Iteration: {} State: {}".format(t, RSTATE))
    
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    
    
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
       
        # skipping perception
        p_obj = targetObj.getPosition()   
        r_obj = targetObj.getQuaternion()
        obj.setPosition(p_obj)
        obj.setQuaternion(r_obj)

        V.recopyMeshes(C)
        V.setConfiguration(C)
        
        
    if RSTATE == CLOSING_GRIPPER:
        if not is_gripping:
            start_grip_object("R_gripper")
            is_gripping = True
        elif grip_is_finished("R_gripper"):
            RSTATE = GOING_BACK
            is_gripping = False
        q = S.get_q()
    
    elif RSTATE == APPORACHING_OBJECT:
        komo = go_to_object("R_gripperCenter", "object")
        q = update(komo)
        
        diff,_ = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter","object"])
        print(np.linalg.norm(diff))
        if np.linalg.norm(diff) < 0.02:
            RSTATE = CLOSING_GRIPPER
        
    elif RSTATE == GOING_BACK:
        pos = [0,0,1]
        komo = move_to_position(pos)
        q = update(komo)
        
        diff,_ = C.evalFeature(ry.FS.position, ["R_gripperCenter"])
        diff -= np.array(pos)
        print(np.linalg.norm(diff))
        if np.linalg.norm(diff) < 0.1:
            RSTATE = RELEASING_OBJECT
    
    elif RSTATE == RELEASING_OBJECT:
        S.openGripper("R_gripper")
        fall_delay += 1
        if fall_delay > 50:
            RSTATE = APPORACHING_OBJECT
            fall_delay = 0
    
    S.step(q, tau, ry.ControlMode.position)


Iteration: 0 State: 0
0.6819101199365638
Iteration: 1 State: 0
0.5337233261584319
Iteration: 2 State: 0
0.46346188891123025
Iteration: 3 State: 0
0.42428594604186526
Iteration: 4 State: 0
0.39673619130558785
Iteration: 5 State: 0
0.3738343176541567
Iteration: 6 State: 0
0.35307045885184635
Iteration: 7 State: 0
0.33370781873094296
Iteration: 8 State: 0
0.3152097499183339
Iteration: 9 State: 0
0.29761380675288207
Iteration: 10 State: 0
0.2892504495898023
Iteration: 11 State: 0
0.2727499377658552
Iteration: 12 State: 0
0.25708844117319274
Iteration: 13 State: 0
0.24224192373280892
Iteration: 14 State: 0
0.22819998850879047
Iteration: 15 State: 0
0.21495661941738395
Iteration: 16 State: 0
0.20248641078902682
Iteration: 17 State: 0
0.19076237507556926
Iteration: 18 State: 0
0.17977278015716225
Iteration: 19 State: 0
0.16950651735202665
Iteration: 20 State: 0
0.1533163001534313
Iteration: 21 State: 0
0.144534688499328
Iteration: 22 State: 0
0.13638255929753745
Iteration: 23 State: 0
0.12873

0.12440453994704737
Iteration: 285 State: 2
0.11754596065208683
Iteration: 286 State: 2
0.11102039247404695
Iteration: 287 State: 2
0.10473980926077753
Iteration: 288 State: 2
0.09881649417718988
Iteration: 289 State: 3
Iteration: 290 State: 3
Iteration: 291 State: 3
Iteration: 292 State: 3
Iteration: 293 State: 3
Iteration: 294 State: 3
Iteration: 295 State: 3
Iteration: 296 State: 3
Iteration: 297 State: 3
Iteration: 298 State: 3
Iteration: 299 State: 3
Iteration: 300 State: 3
Iteration: 301 State: 3
Iteration: 302 State: 3
Iteration: 303 State: 3
Iteration: 304 State: 3
Iteration: 305 State: 3
Iteration: 306 State: 3
Iteration: 307 State: 3
Iteration: 308 State: 3
Iteration: 309 State: 3
Iteration: 310 State: 3
Iteration: 311 State: 3
Iteration: 312 State: 3
Iteration: 313 State: 3
Iteration: 314 State: 3
Iteration: 315 State: 3
Iteration: 316 State: 3
Iteration: 317 State: 3
Iteration: 318 State: 3
Iteration: 319 State: 3
Iteration: 320 State: 3
Iteration: 321 State: 3
Iteration: 3

Iteration: 608 State: 1
Iteration: 609 State: 1
Iteration: 610 State: 1
Iteration: 611 State: 1
Iteration: 612 State: 1
Iteration: 613 State: 1
Iteration: 614 State: 1
Iteration: 615 State: 1
Iteration: 616 State: 1
Iteration: 617 State: 1
Iteration: 618 State: 1
Iteration: 619 State: 1
Iteration: 620 State: 1
Iteration: 621 State: 1
Iteration: 622 State: 1
Iteration: 623 State: 1
Iteration: 624 State: 1
Iteration: 625 State: 1
Iteration: 626 State: 1
Iteration: 627 State: 1
Iteration: 628 State: 1
Iteration: 629 State: 1
Iteration: 630 State: 1
Iteration: 631 State: 1
Iteration: 632 State: 1
Iteration: 633 State: 1
Iteration: 634 State: 1
Iteration: 635 State: 1
Iteration: 636 State: 1
Iteration: 637 State: 1
Iteration: 638 State: 1
Iteration: 639 State: 1
Iteration: 640 State: 1
Iteration: 641 State: 1
Iteration: 642 State: 1
Iteration: 643 State: 1
Iteration: 644 State: 1
Iteration: 645 State: 1
Iteration: 646 State: 1
Iteration: 647 State: 1
Iteration: 648 State: 1
Iteration: 649 S

0.02822011363684336
Iteration: 846 State: 0
0.02814487714019129
Iteration: 847 State: 0
0.028071199257720024
Iteration: 848 State: 0
0.02799508196702791
Iteration: 849 State: 0
0.027920989524280276
Iteration: 850 State: 0
0.02784596420171426
Iteration: 851 State: 0
0.02777309029388727
Iteration: 852 State: 0
0.027698355291050792
Iteration: 853 State: 0
0.027625500447593373
Iteration: 854 State: 0
0.027550933941886124
Iteration: 855 State: 0
0.02747764558812372
Iteration: 856 State: 0
0.02740464739105397
Iteration: 857 State: 0
0.027332496502533794
Iteration: 858 State: 0
0.0272600779359384
Iteration: 859 State: 0
0.02718783009152976
Iteration: 860 State: 0
0.027115957706093954
Iteration: 861 State: 0
0.027044491285762064
Iteration: 862 State: 0
0.026972191489832098
Iteration: 863 State: 0
0.02690193296762806
Iteration: 864 State: 0
0.026831520634198594
Iteration: 865 State: 0
0.026761382417208265
Iteration: 866 State: 0
0.026690809791448732
Iteration: 867 State: 0
0.02662135352671874
I

Iteration: 1048 State: 1
Iteration: 1049 State: 1
Iteration: 1050 State: 1
Iteration: 1051 State: 1
Iteration: 1052 State: 1
Iteration: 1053 State: 1
Iteration: 1054 State: 1
Iteration: 1055 State: 1
Iteration: 1056 State: 1
Iteration: 1057 State: 1
Iteration: 1058 State: 1
Iteration: 1059 State: 1
Iteration: 1060 State: 1
Iteration: 1061 State: 1
Iteration: 1062 State: 1
Iteration: 1063 State: 1
Iteration: 1064 State: 1
Iteration: 1065 State: 1
Iteration: 1066 State: 1
Iteration: 1067 State: 1
Iteration: 1068 State: 1
Iteration: 1069 State: 1
Iteration: 1070 State: 1
Iteration: 1071 State: 1
Iteration: 1072 State: 1
Iteration: 1073 State: 1
Iteration: 1074 State: 1
Iteration: 1075 State: 1
Iteration: 1076 State: 1
Iteration: 1077 State: 1
Iteration: 1078 State: 1
Iteration: 1079 State: 1
Iteration: 1080 State: 1
Iteration: 1081 State: 1
Iteration: 1082 State: 1
Iteration: 1083 State: 1
Iteration: 1084 State: 1
Iteration: 1085 State: 1
Iteration: 1086 State: 1
Iteration: 1087 State: 1


...

0.03513003773198709
Iteration: 9085 State: 0
0.03513473734220227
Iteration: 9086 State: 0
0.03513874751758174
Iteration: 9087 State: 0
0.03514230358583954
Iteration: 9088 State: 0
0.035206442772190746
Iteration: 9089 State: 0
0.03530404682681842
Iteration: 9090 State: 0
0.035185812313939586
Iteration: 9091 State: 0
0.035152560953843184
Iteration: 9092 State: 0
0.0351438063397152
Iteration: 9093 State: 0
0.03514251033500797
Iteration: 9094 State: 0
0.03514456301931402
Iteration: 9095 State: 0
0.035147437023493926
Iteration: 9096 State: 0
0.03520198961230973
Iteration: 9097 State: 0
0.0351503210442116
Iteration: 9098 State: 0
0.03513784249632855
Iteration: 9099 State: 0
0.03517943370235349
Iteration: 9100 State: 0
0.0350949547216863
Iteration: 9101 State: 0
0.03507173746764648
Iteration: 9102 State: 0
0.03506682907751465
Iteration: 9103 State: 0
0.03526145226936643
Iteration: 9104 State: 0
0.03516447204943978
Iteration: 9105 State: 0
0.035238189632041075
Iteration: 9106 State: 0
0.035147

0.03514417899847477
Iteration: 9269 State: 0
0.03512280341807173
Iteration: 9270 State: 0
0.03511955685028143
Iteration: 9271 State: 0
0.035265319073904844
Iteration: 9272 State: 0
0.0352087251651038
Iteration: 9273 State: 0
0.03519470939086522
Iteration: 9274 State: 0
0.03519092976186593
Iteration: 9275 State: 0
0.035191332372494194
Iteration: 9276 State: 0
0.03519286847518442
Iteration: 9277 State: 0
0.03519410670273512
Iteration: 9278 State: 0
0.03519620378210072
Iteration: 9279 State: 0
0.035198454787928865
Iteration: 9280 State: 0
0.035201066764576766
Iteration: 9281 State: 0
0.03520286424235647
Iteration: 9282 State: 0
0.035252802363093395
Iteration: 9283 State: 0
0.03519662759673588
Iteration: 9284 State: 0
0.03517885649336384
Iteration: 9285 State: 0
0.03517399828573693
Iteration: 9286 State: 0
0.0351730931374348
Iteration: 9287 State: 0
0.03517518431047468
Iteration: 9288 State: 0
0.03523522009322917
Iteration: 9289 State: 0
0.03517745425816697
Iteration: 9290 State: 0
0.03516

0.035070479199237864
Iteration: 9452 State: 0
0.035067128068069366
Iteration: 9453 State: 0
0.03506929847077105
Iteration: 9454 State: 0
0.03507376177449841
Iteration: 9455 State: 0
0.035108307780101274
Iteration: 9456 State: 0
0.03506391920200184
Iteration: 9457 State: 0
0.03511493357747751
Iteration: 9458 State: 0
0.03516509173355438
Iteration: 9459 State: 0
0.03506300740652065
Iteration: 9460 State: 0
0.03503620116678515
Iteration: 9461 State: 0
0.03503099963322688
Iteration: 9462 State: 0
0.0350337895987482
Iteration: 9463 State: 0
0.035038719419593745
Iteration: 9464 State: 0
0.03504515459827285
Iteration: 9465 State: 0
0.03505085739395811
Iteration: 9466 State: 0
0.03505665950256905
Iteration: 9467 State: 0
0.03506184906118203
Iteration: 9468 State: 0
0.0350682189989892
Iteration: 9469 State: 0
0.03507222932894136
Iteration: 9470 State: 0
0.035078772962252536
Iteration: 9471 State: 0
0.035084152871760185
Iteration: 9472 State: 0
0.035086117977844694
Iteration: 9473 State: 0
0.035

0.03518478889847182
Iteration: 9635 State: 0
0.03519844094596051
Iteration: 9636 State: 0
0.03517070815538503
Iteration: 9637 State: 0
0.03516342373397951
Iteration: 9638 State: 0
0.03516268489051871
Iteration: 9639 State: 0
0.03516430121392688
Iteration: 9640 State: 0
0.035166233219749696
Iteration: 9641 State: 0
0.03516584198209248
Iteration: 9642 State: 0
0.03516915460165136
Iteration: 9643 State: 0
0.03517169214493836
Iteration: 9644 State: 0
0.0351736313683869
Iteration: 9645 State: 0
0.03517519946757007
Iteration: 9646 State: 0
0.03517702287493744
Iteration: 9647 State: 0
0.03517875058623053
Iteration: 9648 State: 0
0.03517848416069097
Iteration: 9649 State: 0
0.03520323218813472
Iteration: 9650 State: 0
0.03517621695764765
Iteration: 9651 State: 0
0.035164529928348544
Iteration: 9652 State: 0
0.035163338276304995
Iteration: 9653 State: 0
0.035163792069858674
Iteration: 9654 State: 0
0.03516484333679899
Iteration: 9655 State: 0
0.03516654264030495
Iteration: 9656 State: 0
0.03516

0.03506397036702978
Iteration: 9819 State: 0
0.03503966827056358
Iteration: 9820 State: 0
0.03503596538526139
Iteration: 9821 State: 0
0.035039250499330994
Iteration: 9822 State: 0
0.03498183593274866
Iteration: 9823 State: 0
0.03499086228176492
Iteration: 9824 State: 0
0.03514933593398039
Iteration: 9825 State: 0
0.035062278298783695
Iteration: 9826 State: 0
0.035016765167593306
Iteration: 9827 State: 0
0.03500436997194178
Iteration: 9828 State: 0
0.035003028186543376
Iteration: 9829 State: 0
0.035051680101629956
Iteration: 9830 State: 0
0.035003847037319805
Iteration: 9831 State: 0
0.03498889941535139
Iteration: 9832 State: 0
0.03499083978171154
Iteration: 9833 State: 0
0.03508825720456182
Iteration: 9834 State: 0
0.03511095845745793
Iteration: 9835 State: 0
0.03503029889678486
Iteration: 9836 State: 0
0.03500570701867146
Iteration: 9837 State: 0
0.03499729273814512
Iteration: 9838 State: 0
0.03500095262327254
Iteration: 9839 State: 0
0.035007148891584315
Iteration: 9840 State: 0
0.0